In [45]:
import random
import time
import warnings
import tqdm
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import models
from utils import *
from dataset.imbalance_cifar import ImbalanceCIFAR10, ImbalanceCIFAR100
from sklearn.manifold import TSNE
from matplotlib import cm
%matplotlib inline

%load_ext autoreload
%autoreload 2

# model_path = 'checkpoint/cifar10_resnet32_Focal_None_exp_0.01_stage1/ckpt.best.pth.tar'
# model_path = 'checkpoint/cifar10_resnet32_LDAM_None_exp_0.01_stage1/ckpt.best.pth.tar'
model_path = 'checkpoint/cifar10_resnet32_LDAM_Resample_exp_0.01_stage2/ckpt.best.pth.tar'
# model_path = 'checkpoint/cifar10_resnet32_LDAM_DRW_exp_0.01_stage2/ckpt.best.pth.tar'


# model_path = 'checkpoint/cifar10_resnet32_LDAM_None_exp_0.01_stage1/ckpt.pth.tar'
# model_path = 'checkpoint/cifar10_resnet32_LDAM_Reweight_exp_0.01_stage2_freeze/ckpt.pth.tar'
model_path = "checkpoint/cifar10_resnet32_LDAM_DRW_exp_0.01_stage2_upperbound_freeze/ckpt.best.pth.tar"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
use_norm = True if 'LDAM' in model_path else False
model = models.__dict__['resnet32'](num_classes=10, use_norm=use_norm)

checkpoint = torch.load(model_path, map_location=torch.device(f'cuda:0'))
model.load_state_dict(checkpoint['state_dict'])


model.cuda()

ResNet_s(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affin

In [47]:
for name, param in model.fc.named_parameters():
    if 'weight' in name:
        ax = 0 if 'LDAM' in model_path else 1
        norms = np.linalg.norm(param.detach().cpu().numpy(), axis=ax)
print(f'2-norms for each class:')
print(norms)

2-norms for each class:
[5.7583327 4.3362346 5.4050956 5.4535894 4.3272285 4.516941  3.917643
 3.5760722 3.415918  3.1646748]
